In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-200rfq-30cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

15329

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

accept_count = 0
accept_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'ACCEPT':
        accept_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        accept_count += 1   


print(f"#ACCEPTs =  {accept_count}")

#ACCEPTs =  200


In [5]:
df_test = grouped_data.get_group(accept_ids[37])
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
41902,received_tx,107,ACCEPT,f43d5c0857d9157d5967e0675e178b583fa6afb7c1186c...,None,1.670205e+09
41904,before_tendermint,122,ACCEPT,f43d5c0857d9157d5967e0675e178b583fa6afb7c1186c...,None,1.670205e+09
41906,check_tx,147,ACCEPT,f43d5c0857d9157d5967e0675e178b583fa6afb7c1186c...,None,1.670205e+09
41910,check_tx,257,ACCEPT,f43d5c0857d9157d5967e0675e178b583fa6afb7c1186c...,None,1.670205e+09
41911,deliver_tx,320,ACCEPT,f43d5c0857d9157d5967e0675e178b583fa6afb7c1186c...,None,1.670205e+09
41912,end_block,322,ACCEPT,f43d5c0857d9157d5967e0675e178b583fa6afb7c1186c...,None,1.670205e+09
41913,commit_tx,461,ACCEPT,f43d5c0857d9157d5967e0675e178b583fa6afb7c1186c...,None,1.670205e+09


## without RETURN 

In [6]:
time_all_committed_accepts = {}
time_passed_accepts = []
validated_time = []
accepted_time = []
committed_time = []

accept_underfit = []
accept_underfit_dict = {}

accept_overfit = []
accept_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            accept_underfit.append(temp_list)
            accept_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            accept_overfit.append(actual_time)
            accept_overfit_dict[tx_id[i]] = temp_list

## with RETURNs

In [7]:


accept_return_count = {}

time_id_return = {}
for i in range(0, len(tx_id)):
    temp_list = []
    return_count = 0
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        for j in range(0, len(data)):
            if data['Event'][j] == "commit_tx" and tx_id[i] == data['Accept_id'][j]:
                temp_list.append(data["Time"][j])
                return_count += 1
        accept_return_count[tx_id[i]] = return_count
        

In [8]:
accept_return_count

{'023ce0aa49a2e78035fcc39f61ec0a404f4e4816c6198a1257d1fa2325f50bc4': 25,
 '02744492ccba91d50e35525f7267f44ec01ab6d22a258e0a154386429398a526': 25,
 '032850d9dfabf7a2d41c2bc9b7f571b87f058ec8417c618f639186dc2d28b0c7': 25,
 '0335f35000742a15fb01147eb1304288634213a9335ec41f1b632f644aaab2c6': 25,
 '060313006a60472cce118af147c1bc86b3b60059c7dbb39a2ffa3fc67360a13c': 25,
 '0835ba5f46193a1b0bc613f23ad929f001b060fccb1ae202a7acfaab6f1d8bbb': 25,
 '0aa8cb17f19f078ba18787aa6540cc4ce312d82d9cc39418870333312ba213cd': 25,
 '0c450be1e9d346bfce660c347fbb77179458f86311c42a43a9cde713dc7c1b10': 25,
 '0d8185c1c802fb72aece30469f828c304683d5ef4fea30aa1057655d3c615219': 25,
 '0d9de15d989f051aaacf623b129152d529ac5c702fa59b0a8a285b70f2269f65': 25,
 '0eb4c22a1a4e67fcf58d000e680bc7859bf1dc12fcb0639cd769fe107281d884': 25,
 '0f64e05e7ac3f158491b007740a0cac4b3b622e16537b64f81fa29cd30f4f2ea': 25,
 '11b7062bf2ab45117b7c64949a1910f6f627dd65b0f47cb9162ac38fdcdd7a33': 25,
 '128dce7b423945eeea281ddc9a78b74be6f60c84336fe8dcb

In [9]:
time_all_committed_accepts

{'023ce0aa49a2e78035fcc39f61ec0a404f4e4816c6198a1257d1fa2325f50bc4': 402,
 '02744492ccba91d50e35525f7267f44ec01ab6d22a258e0a154386429398a526': 347,
 '032850d9dfabf7a2d41c2bc9b7f571b87f058ec8417c618f639186dc2d28b0c7': 411,
 '0335f35000742a15fb01147eb1304288634213a9335ec41f1b632f644aaab2c6': 394,
 '060313006a60472cce118af147c1bc86b3b60059c7dbb39a2ffa3fc67360a13c': 340,
 '0835ba5f46193a1b0bc613f23ad929f001b060fccb1ae202a7acfaab6f1d8bbb': 335,
 '0aa8cb17f19f078ba18787aa6540cc4ce312d82d9cc39418870333312ba213cd': 427,
 '0c450be1e9d346bfce660c347fbb77179458f86311c42a43a9cde713dc7c1b10': 390,
 '0d8185c1c802fb72aece30469f828c304683d5ef4fea30aa1057655d3c615219': 365,
 '0d9de15d989f051aaacf623b129152d529ac5c702fa59b0a8a285b70f2269f65': 363,
 '0eb4c22a1a4e67fcf58d000e680bc7859bf1dc12fcb0639cd769fe107281d884': 388,
 '0f64e05e7ac3f158491b007740a0cac4b3b622e16537b64f81fa29cd30f4f2ea': 401,
 '11b7062bf2ab45117b7c64949a1910f6f627dd65b0f47cb9162ac38fdcdd7a33': 199,
 '128dce7b423945eeea281ddc9a78b74be6f6

In [10]:
len(time_passed_accepts)

198

In [11]:
len(accept_overfit)

195

In [12]:
stat.mean(time_passed_accepts)

386.979797979798

In [13]:
stat.mean(accept_overfit)

389.51794871794874

In [14]:
# Analysis of CREATE Transaction
accept_mean = "{:.2f}".format(stat.mean(time_passed_accepts))
accept_median = stat.median(time_passed_accepts)
accept_max = max(time_passed_accepts)
accept_min = min(time_passed_accepts)
accept_std_dev = "{:.2f}".format(stat.stdev(time_passed_accepts))

print(accept_mean)
print(accept_median)
print(accept_max)
print(accept_min)
print(accept_std_dev)
print(f"total accepts = {len(time_passed_accepts)}")

accept_dict = {
               'min': accept_min,
               'max': accept_max,
               'average': accept_mean,
               'median' : accept_median,
               'standard_deviation' : accept_std_dev,
               '# of txs' : len(time_passed_accepts)}

386.98
384.0
1026
199
64.70
total accepts = 198


In [15]:
results = pd.DataFrame(accept_dict, index = ['ACCEPT'])
results.to_csv('accept_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
ACCEPT,199,1026,386.98,384.0,64.70,198
